<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ch08で使用する共通の前処理をまとめる
- wv = KeyedVectors.load_word2vec_format(...)
- token2id / id2token / E の構築
- filter_token, row_to_ids などの前処理関数

In [1]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 35.5 MB/s eta 0:00:00


In [13]:
from gensim.models import KeyedVectors
from google.colab import drive
import numpy as np
import pandas as pd
import torch

In [9]:
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
wv_src = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz' # gzipはデータの中の繰り返しパターンを短い表現に置き換える
wv = KeyedVectors.load_word2vec_format(wv_src, binary=True) # binary=Trueでバイナリ形式（0, 1のバイト列）で対応できるようにする

In [12]:
def filter_token(row):
  tokens = str(row['sentence']).split() # sentenceが欠損している場合を考えてstrで文字列変換し、NaNを空文字にしている
  input_ids = []
  not_found_count = 0
  for token in tokens:
    if token in token2id:
      input_ids.append(token2id[token])
  return input_ids

In [11]:
def row_to_ids(row):
  input_ids = filter_token(row)

  if len(input_ids) == 0:
    return None

  return {
      "text": row["sentence"],
      "label": torch.tensor([row["label"]], dtype=torch.float32),
      "input_ids": torch.tensor(input_ids, dtype=torch.long)
  }

In [15]:
V = len(wv) + 1 # +1は<PAD>分
print('語彙数:', V)

d = wv.vector_size
print('埋め込み次元数', d)

print(wv.vectors.shape) # GoogleNews-vectors-negative300の埋め込み行列
print(wv.vectors.nbytes / 1024**3) # 使用メモリ数
E = torch.zeros((V, d), dtype=torch.float32) # PyTorchのニューラルネットが期待している型がfloat32らしい。今回は学習範囲外
token2id = {"<PAD>": 0}
id2token = ["<PAD>"]

語彙数: 3000001
埋め込み次元数 300
(3000000, 300)
3.3527612686157227


In [16]:
for i, token in enumerate(wv.index_to_key, start=1): # すでに0のindexはPADに割り当てているので1から開始する
  token2id[token] = i
  id2token.append(token)
  E[i] = torch.tensor(wv[token]) # from_numpyはnumpy配列をPyTorchテンソル型に変換する関数（torch.Tensor）